# Ping miner pools with stratum-ping
- https://github.com/2miners/stratum-ping
- https://www.reddit.com/r/EtherMining/comments/o7l0qv/stratum_ping_times_for_your_location/

Used to check if their is problem with specific pool or port.
Check extra if firewall is not correct.

When their is proxy problem test
- https://stratumproxy.io/

In [21]:
# Download stratum-ping
# !wget https://github.com/2miners/stratum-ping/releases/download/v1.0.0/stratum-ping-linux-x64.tar.gz
# !tar -xf stratum-ping-linux-x64.tar.gz

In [22]:
import subprocess
import re

In [23]:
# check startum-ping is downloaded
!./stratum-ping -h

Usage of ./stratum-ping:
  -6	use ipv6
  -c int
    	stop after <count> replies (default 5)
  -p string
    	pass (default "x")
  -t string
    	stratum type: stratum1, stratum2 (default "stratum2")
  -tls
    	use TLS
  -u string
    	login (default "0x63a14c53f676f34847b5e6179c4f5f5a07f0b1ed")


In [24]:
# run one test
!./stratum-ping -c 2 stratum-ethw.antpool.com:443

PING stratum stratum-ethw.antpool.com (172.65.231.105) port 443
stratum-ethw.antpool.com (172.65.231.105): seq=0, time=669.4423ms
stratum-ethw.antpool.com (172.65.231.105): seq=1, time=629.9645ms

--- stratum-ethw.antpool.com ping statistics ---
2 packets transmitted, 2 received, 0% packet loss, time 3.3418231s
min/avg/max = 629.9645ms, 649.7034ms, 669.4423ms


In [25]:
for i in [1, 2]:
    print(f'\n\n---------- START {i} -----------------')
    !./stratum-ping -c 1 eth.f2pool.com:6688




---------- START 1 -----------------


PING stratum eth.f2pool.com (172.65.212.50) port 6688
eth.f2pool.com (172.65.212.50): seq=0, time=107.0641ms

--- eth.f2pool.com ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 1.1339904s
min/avg/max = 107.0641ms, 107.0641ms, 107.0641ms


---------- START 2 -----------------
PING stratum eth.f2pool.com (172.65.212.50) port 6688
eth.f2pool.com (172.65.212.50): seq=0, time=85.6629ms

--- eth.f2pool.com ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 1.1099598s
min/avg/max = 85.6629ms, 85.6629ms, 85.6629ms


## Servers

In [26]:
rvn_servers = [
    ["Cruxpool", "Europe", "stratum+tcp://eu.cruxpool.com:9999"],
    ["HeroMiners", "Global", "stratum+tcp://ravencoin.herominers.com:10240"],
    ["HeroMiners", "Global", "stratum+tcp://fi.ravencoin.herominers.com:1140"],
    ["BSOD.PW", "Europe", "stratum+tcp://eu.bsod.pw:2640"],
    ["MinerMore", "Europe", "stratum+tcp://eu.rvn.minermore.com:4501"],
    ["Suprnova", "Global", "stratum+tcp://rvn.suprnova.cc:8888"],
    ["Kryptex", "Global", "stratum+tcp://rvn.kryptex.network:7777"],
    ["Kryptex", "Global", "stratum+tcp://rvn.kryptex.network:8888"],
    ["2miners.com", "Global", "stratum+tcp://rvn.2miners.com:6060"],
    ["RavenMiner", "Global", "stratum+tcp://stratum.ravenminer.com:3838"],
    ["Nanopool", "Europe 1", "stratum+tcp://rvn-eu1.nanopool.org:12222"],
    ["Nanopool", "Europe 1 (SSL)", "stratum+ssl://rvn-eu1.nanopool.org:12433"],
    ["K1Pool", "Europe", "stratum+tcp://eu.rvn.k1pool.com:7861"]
]


In [27]:
eth_servers = [
    ["2Miners", "Global", "stratum+tcp://eth.2miners.com:2020"],
    ["HelloMiner", "EU1", "stratum+tcp://eth-eu1.hellominer.com:1100"],
    ["Unmineable", "Global", "stratum+tcp://ethash.unmineable.com:3333"],
    ["F2Pool", "Global", "stratum+tcp://etc.f2pool.com:8118"],
    ["F2Pool", "Global", "stratum+tcp://eth.f2pool.com:6688"]
]


In [28]:
test_servers = [
    # Adding AntPool servers
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:25"],
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:3333"],
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:443"],
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:445"],
    # Adding Ravencoin server on AntPool
    ["AntPool", "RVN", "stratum+tcp://rvn.antpool.com:8003"],
    # Adding Kaspa server on AntPool
    ["AntPool", "KAS", "stratum+tcp://kas.antpool.com:445"],
    # Adding EthereumWorld servers on AntPool
    ["AntPool", "ETHW", "stratum+tcp://stratum-ethw.antpool.com:25"],
    ["AntPool", "ETHW", "stratum+tcp://stratum-ethw.antpool.com:443"],
    # Adding Poolin servers for EthereumWorld
    ["Poolin", "ETHW", "stratum+tcp://ethw.ss.poolin.one:443"],
    ["Poolin", "ETHW", "stratum+tcp://ethw.ss.poolin.one:1883"],
    ["Poolin", "ETHW", "stratum+tcp://ethw.ss.poolin.one:700"],
    ["Xmrig", "Global", "stratum+tcp://donate.ssl.xmrig.com:443"]
]

In [29]:
# combine all servers
all_servers = test_servers + rvn_servers + eth_servers

## Check all servers

In [30]:
def successful_transmission(text):
    # Regex to find the number of packets received
    packets_received_pattern = r", (\d+) received,"
    packets_received_match = re.search(packets_received_pattern, text)
    if packets_received_match:
        return int(packets_received_match.group(1))
    else:
        return 0

def run_stratum_ping(server):
    # The command to run, assuming stratum-ping is in the current working directory or in the PATH
    command = ['./stratum-ping', '-c', '1', server]

    # Execute the command
    try:
        # If you want to capture the output
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # # Print the standard output
        # print(result.stdout)

        # # Check if there were any errors
        # if result.stderr:
        #     print("Errors:\n", result.stderr)
    except subprocess.CalledProcessError as e:
        print("An error occurred while trying to execute stratum-ping:", str(e))
    except FileNotFoundError:
        print("stratum-ping executable not found. Ensure it's in the current working directory or its path is correctly set.")

    return [successful_transmission(result.stdout), server, result.stdout]


In [31]:
def check_stratum_ping(servers):
    for i, server in enumerate(servers):
        print(f'\n\n---------- START {i} -----------------')
        res = run_stratum_ping(server)
        print(res)

In [32]:
# sample test
# res = check_stratum_ping(['donate.ssl.xmrig.com:443'])
check_stratum_ping(['eth.f2pool.com:6688', 'rvn-asia1.nanopool.org:12433'])



---------- START 0 -----------------


[1, 'eth.f2pool.com:6688', 'PING stratum eth.f2pool.com (172.65.212.50) port 6688\neth.f2pool.com (172.65.212.50): seq=0, time=96.4922ms\n\n--- eth.f2pool.com ping statistics ---\n1 packets transmitted, 1 received, 0% packet loss, time 1.1243933s\nmin/avg/max = 96.4922ms, 96.4922ms, 96.4922ms\n']


---------- START 1 -----------------
[0, 'rvn-asia1.nanopool.org:12433', 'PING stratum rvn-asia1.nanopool.org (51.79.145.202) port 12433\nrvn-asia1.nanopool.org (51.79.145.202): seq=0, EOF\n\n--- rvn-asia1.nanopool.org ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 1.5294404s\n']


In [33]:
def check_all_servers(servers):
    results = []
    # strip protocol from adress
    server_addresses = [server[2].replace('stratum+ssl://', '').replace('stratum+tcp://', '') for server in servers]
    for server in server_addresses:
        res = run_stratum_ping(server)
        print(res)
        results.append(res)

    return results

In [34]:
res = check_all_servers(all_servers)
# res = check_all_servers(eth_servers)
# res = check_all_servers(rvn_servers)
# res = check_all_servers(test_servers)

[0, 'stratum.antpool.com:25', 'PING stratum stratum.antpool.com (172.65.209.66) port 25\nstratum.antpool.com (172.65.209.66): seq=0, dial tcp4 172.65.209.66:25: connect: connection timed out\n\n--- stratum.antpool.com ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 2m11.6793195s\n']
[0, 'stratum.antpool.com:3333', 'PING stratum stratum.antpool.com (172.65.209.66) port 3333\nstratum.antpool.com (172.65.209.66): seq=0, read tcp4 172.26.170.100:51028->172.65.209.66:3333: i/o timeout\n\n--- stratum.antpool.com ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 11.0199769s\n']
[0, 'stratum.antpool.com:443', 'PING stratum stratum.antpool.com (172.65.209.66) port 443\nstratum.antpool.com (172.65.209.66): seq=0, read tcp4 172.26.170.100:38262->172.65.209.66:443: i/o timeout\n\n--- stratum.antpool.com ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 11.0196641s\n']
[0, 'stratum.antpool.com:445', 'PING strat